In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

m:\Third Year\Sixth Semester\Projects\NLP-Project\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class semanticEmbedding:
    def __init__(self, model_name='sentence-transformers/all-mpnet-base-v2'):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)

    def mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    def get_embedding(self, sentences):
    # Tokenize sentences
        encoded_input = self.tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            model_output = self.model(**encoded_input)
        # Perform pooling
        sentence_embeddings = self.mean_pooling(model_output, encoded_input['attention_mask'])

        # Normalize embeddings
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        return sentence_embeddings.detach().numpy()

In [3]:
# creating an instance of the class
model = semanticEmbedding()

In [4]:
import faiss 
import numpy as np
import pickle

In [5]:
class FaissIdx:

    def __init__(self, model, dim=768):
        self.index = faiss.IndexFlatIP(dim) # need to load the pickle model in the final file
        # self.index = faiss
        # Maintaining the document data
        self.doc_map = dict()
        self.model = model
        self.ctr = 0

    def add_doc(self, document_text):
        self.index.add(self.model.get_embedding(document_text))
        self.doc_map[self.ctr] = document_text # store the original document text
        self.ctr += 1

    def search_doc(self, query, k=3):
        D, I = self.index.search(self.model.get_embedding(query), k)
        return [{self.doc_map[idx]: score} for idx, score in zip(I[0], D[0]) if idx in self.doc_map]
    def save_index(self, index_filename, doc_map_filename):
        # Save Faiss index to file
        faiss.write_index(self.index, index_filename)

        # Save document map to file using pickle
        with open(doc_map_filename, 'wb') as f:
            pickle.dump(self.doc_map, f)

    def load_index(self, index_filename, doc_map_filename):
        # Load Faiss index from file
        self.index = faiss.read_index(index_filename)

        # Load document map from file using pickle
        with open(doc_map_filename, 'rb') as f:
            self.doc_map = pickle.load(f)

In [6]:
index = FaissIdx(model)

In [7]:
import pandas as pd

In [8]:
df = pd.read_csv('scraped.csv')

In [9]:
total = df.shape[0]
total

869

In [10]:
from IPython.display import clear_output

In [11]:
for idx, row in df.iterrows():
    index.add_doc(row['content'])
    print(((idx)/total)*100)
    if idx%10==0:
        clear_output(wait=True)

index.save_index('index.bin', 'doc_map.pkl')

99.07940161104717
99.1944764096663
99.30955120828538
99.42462600690449
99.5397008055236
99.6547756041427
99.76985040276179
99.88492520138091
